In [1]:
import pandas as pd
from scipy import stats
from matplotlib import cm, colors
from mpl_toolkits.axes_grid1 import ImageGrid
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

2023-05-28 22:02:35.616613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 22:02:35.764794: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-28 22:02:36.675073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/jet/home/jlin96/.conda/envs/tf2/lib/
2023-05-28 22:02:36.675234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plug

In [2]:
# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

In [3]:
!pwd

/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy


In [4]:
# change these for your HPC and repo path
data_path = '/ocean/projects/atm200007p/walrus/for_jerry/train/'
norm_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/norm_factors/'
grid_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/test_data/E3SM-MMF_ne4_grid-info.orig.nc'
save_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/'

In [5]:
# new dataset generator function
# that has new options (latlim, lonlim)

mli_mean = xr.open_dataset(norm_path + 'mli_mean.nc')
mli_min = xr.open_dataset(norm_path + 'mli_min.nc')
mli_max = xr.open_dataset(norm_path + 'mli_max.nc')
mlo_scale = xr.open_dataset(norm_path + 'mlo_scale.nc')
ne4_grid_info = xr.open_dataset(grid_path)

def load_nc_dir_with_generator_test(filelist:list, latlim=[-999,999], lonlim=[-999,999]):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]
            ds = ds.merge(ne4_grid_info[['lat','lon']])
            ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
            ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            dso = dso.merge(ne4_grid_info[['lat','lon']])
            dso = dso.where((dso['lat']>latlim[0])*(dso['lat']<latlim[1]),drop=True)
            dso = dso.where((dso['lon']>lonlim[0])*(dso['lon']<lonlim[1]),drop=True)
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalizatoin, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

In [6]:
# validation dataset for HPO
stride_sample = 6
f_mli1 = glob.glob(data_path + '*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc')
f_mli2 = glob.glob(data_path + '*/E3SM-MMF.mli.0008-1[012]-*-*.nc')
f_mli3 = glob.glob(data_path + '*/E3SM-MMF.mli.0009-01-*-*.nc')
f_mli_data = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli_data = f_mli_data[::stride_sample]
print(f'#files: {len(f_mli_data)}')

# creating numpy array defeats the purpose of tf Dataset pipeline,
# but, just doing it here for quick sanity check.
tds_data = load_nc_dir_with_generator_test(f_mli_data)
work = list(tds_data.as_numpy_iterator())
x_true = np.concatenate([ work[k][0] for k in range(len(work)) ])
y_true = np.concatenate([ work[k][1] for k in range(len(work)) ])

with open(save_path + "val_input_stride6.npy", 'wb') as f:
    np.save(f, np.float32(x_true))
with open(save_path + "val_target_stride6.npy", 'wb') as f:
    np.save(f, np.float32(y_true))

#files: 4380


2023-05-28 22:03:11.514438: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/jet/home/jlin96/.conda/envs/tf2/lib/
2023-05-28 22:03:11.518777: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-28 22:03:11.519174: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (r228.ib.bridges2.psc.edu): /proc/driver/nvidia/version does not exist
2023-05-28 22:03:11.613997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
print("finished")

finished


In [8]:
!pwd

/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy
